In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK075423,Karma Resorts,Other,Past Guests,Karma Kandara,PF317091,Mr.,Ned Paul Ben,Blackman,Male,...,2025-09-28 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KK078708,Karma Resorts,Other,Past Guests,Karma Kandara,PF282014,Miss,Erin Lee,Jude,Female,...,2025-09-28 00:00:00,6,HOUSE_USE,V_OWNER_NRB,COMP,6,3,0,Email and Phone,NO
2,KK079478,Karma Resorts,Other,Past Guests,Karma Kandara,PF282014,Miss,Erin Lee,Jude,Female,...,2025-09-28 00:00:00,6,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
3,KK069419,Karma Resorts,Other,Past Guests,Karma Kandara,PF302986,Mr.,Kyal Ronald,Grut,Male,...,2025-10-03 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KK080411,Karma Resorts,Other,Past Guests,Karma Kandara,PF330747,Miss,Natalie Gwendoline,Poggioli,Female,...,2025-09-30 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,KSM017114,Karma Resorts,Other,Past Guests,Karma St Martins,PF330978,Ms,Hillary,Bartlett,Female,...,2025-10-02 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Phone Only,NO
58,KK076050,Karma Resorts,Other,Past Guests,Karma Kandara,PF318613,Miss,Robyn Elizabeth,Ducharme,Female,...,2025-09-28 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
59,KLM007341,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF293303,Mr,Scott,Griffin,Male,...,2025-10-01 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
60,KB053695,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331120,Mrs,Theresa,Walter,Female,...,2025-09-28 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
2,KK079478,Karma Resorts,Other,Past Guests,Karma Kandara,PF282014,Miss,Erin Lee,Jude,Female,...,2025-09-28 00:00:00,6,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK075423,Karma Resorts,Other,Past Guests,Karma Kandara,PF317091,Mr.,Ned Paul Ben,Blackman,Male,...,2025-09-28 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KK078708,Karma Resorts,Other,Past Guests,Karma Kandara,PF282014,Miss,Erin Lee,Jude,Female,...,2025-09-28 00:00:00,6,HOUSE_USE,V_OWNER_NRB,COMP,6,3,0,Email and Phone,NO
2,KK069419,Karma Resorts,Other,Past Guests,Karma Kandara,PF302986,Mr.,Kyal Ronald,Grut,Male,...,2025-10-03 00:00:00,1,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
3,KK080411,Karma Resorts,Other,Past Guests,Karma Kandara,PF330747,Miss,Natalie Gwendoline,Poggioli,Female,...,2025-09-30 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
4,KK068814,Karma Resorts,Other,Past Guests,Karma Kandara,PF301687,Dr,Juliette,Tobias,Unknown,...,2025-10-03 00:00:00,1,WEDDING,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,KSM017114,Karma Resorts,Other,Past Guests,Karma St Martins,PF330978,Ms,Hillary,Bartlett,Female,...,2025-10-02 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Phone Only,NO
57,KK076050,Karma Resorts,Other,Past Guests,Karma Kandara,PF318613,Miss,Robyn Elizabeth,Ducharme,Female,...,2025-09-28 00:00:00,5,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
58,KLM007341,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF293303,Mr,Scott,Griffin,Male,...,2025-10-01 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
59,KB053695,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331120,Mrs,Theresa,Walter,Female,...,2025-09-28 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
5,KK064789,Karma Resorts,Other,Past Guests,Karma Kandara,PF294292,Mrs,Kloe,Balsley,Female,...,2025-10-01 00:00:00,7,DIGITAL,GHPA,DIGITAL,2,3,0,Email and Phone,NO
22,KB046928,Karma Resorts,Other,Past Guests,Karma Bavaria,PF299070,Frau,Maria,Weinhuber,Female,...,2025-10-06 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
25,KK059535,Karma Resorts,Other,Past Guests,Karma Kandara,PF191033,Miss,Petra Sabine,Maurer,Female,...,2025-09-29 00:00:00,5,HOUSE_USE,GER_OWE,COMP,4,0,0,Email and Phone,NO
39,KK078842,Karma Resorts,Other,Past Guests,Karma Kandara,PF078473,Mr,Philip Seng Whatt,Tan,Male,...,2025-10-01 00:00:00,14,HOUSE_USE,V_OWNER,COMP,1,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK075423,Karma Resorts,Other,Past Guests,Karma Kandara,PF317091,Mr.,Ned Paul Ben,Blackman,Male,...,2025-09-28 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KB053695,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331120,Mrs,Theresa,Walter,Female,...,2025-09-28 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
2,KB049542,Karma Resorts,Other,Past Guests,Karma Bavaria,PF310192,empty,Ulf,Geumann,Unknown,...,2025-09-28 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
3,KB043682,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287005,Mr,Kea,Staff,Male,...,2025-09-28 00:00:00,27,STAFF_PRIV,STAFF,COMP,1,0,0,Email Only,NO
4,KB051751,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331345,Frau,Jennifer,Plöckl,Female,...,2025-09-28 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
5,KB051751,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331356,Frau,Christina,Götz,Female,...,2025-09-28 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
6,KB053355,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329346,Herr,Michael,Buss,Unknown,...,2025-09-28 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
7,KSM017098,Karma Resorts,Other,Past Guests,Karma St Martins,PF330720,Mr,Matt,Simons,Unknown,...,2025-09-28 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
8,KLM006846,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF235376,Mr,Christopher,Colclough,Unknown,...,2025-09-28 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
9,KB051751,Karma Resorts,Other,Past Guests,Karma Bavaria,PF331349,Frau,Veronika,Pollinger,Female,...,2025-09-28 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_19896\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK075423,Karma Resorts,Other,Past Guests,Karma Kandara,PF317091,Mr.,Ned Paul Ben,Blackman,Male,...,2025-09-28,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KB051751,Karma Resorts,Other,Past Guests,Karma Bavaria,PF320790,Frau,Stefanie,Zeller,Unknown,...,2025-09-28,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
2,KK078708,Karma Resorts,Other,Past Guests,Karma Kandara,PF282014,Miss,Erin Lee,Jude,Female,...,2025-09-28,6,HOUSE_USE,V_OWNER_NRB,COMP,6,3,0,Email and Phone,NO
3,KK076050,Karma Resorts,Other,Past Guests,Karma Kandara,PF318613,Miss,Robyn Elizabeth,Ducharme,Female,...,2025-09-28,5,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KSM014805,Karma Resorts,Other,Past Guests,Karma St Martins,PF304538,Miss,Zoe,Phillips,Unknown,...,2025-09-28,0,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
5,KK070684,Karma Resorts,Other,Past Guests,Karma Kandara,PF305539,Miss,Mikayla Rose,Johnson,Female,...,2025-09-28,4,HOUSE_USE,V_OWNER,COMP,8,0,0,Email and Phone,NO
6,KK062739,Karma Resorts,Other,Past Guests,Karma Kandara,PF290460,Miss,Shaylee Rose,Robinson,Female,...,2025-09-28,3,WEDDING,WEDDING,GROUP,10,0,0,Email and Phone,NO
7,KK063160,Karma Resorts,Other,Past Guests,Karma Kandara,PF291272,Miss,Dianne Pamela,Robinson,Female,...,2025-09-28,3,WEDDING,WEDDING,GROUP,2,0,0,Email and Phone,NO
8,KK062987,Karma Resorts,Other,Past Guests,Karma Kandara,PF290841,Mrs,Tahnee,Mclaren,Female,...,2025-09-28,3,WEDDING,WEDDING,GROUP,2,2,0,Email and Phone,NO
9,KK063149,Karma Resorts,Other,Past Guests,Karma Kandara,PF291240,Mrs,Lorraine,Canham,Female,...,2025-09-28,3,DIGITAL,WEBSITE,DIGITAL,4,0,0,Email and Phone,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 62
Jumlah Data Yang Duplicate : 1
Jumlah Data Setelah Hapus Duplicate : 61
Jumlah Data Yang Termasuk Member: 4
Jumlah Data Setelah Hapus Member : 57
Jumlah Data Akhir : 57


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)